<a href="https://colab.research.google.com/github/corrin/opendental/blob/main/ReadingGoPuzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import random
import copy
import os


In [10]:
base_path = "drive/MyDrive/auto_puzzle/sgf"

if not os.path.exists(base_path):
  os.makedirs(base_path)


In [40]:

class GoBoard:
    def __init__(self, size=19):
        self.size = size
        self.board = np.full((size, size), '.', dtype=str)
        self.atari_position = None

    def place_stone(self, row, col, color):
        if self.board[row, col] == '.':
            self.board[row, col] = color
            if color == 'W' and self.atari_position is None:  # Track initial White stone in atari
                self.atari_position = (row, col)
            return True
        return False

    def random_fill(self, num_stones):
        filled = 0
        while filled < num_stones:
            row, col = np.random.randint(0, self.size, size=2)
            if self.place_stone(row, col, np.random.choice(['B', 'W'])):
                filled += 1


    def place_stone_in_atari(self):
        placed = False
        while not placed:
            # Place the white stone anywhere on the board
            row, col = np.random.randint(0, self.size, size=2)
            if self.place_stone(row, col, 'W'):
                directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right
                np.random.shuffle(directions)  # Shuffle directions

                # Try placing black stones in shuffled order and check for exactly one liberty
                for dr, dc in directions:
                    nr, nc = row + dr, col + dc
                    # Check if the new position is within board boundaries
                    if 0 <= nr < self.size and 0 <= nc < self.size:
                        self.place_stone(nr, nc, 'B')
                        if self.count_liberties(row, col) == 1:
                            placed = True
                            break
            else:
                # Reset the board if the loop completes without breaking (white stone not in correct atari)
                self.reset_board()  # Assumes a method to reset the board state

    def count_liberties(self, row, col):
        visited, stack = set(), [(row, col)]
        liberties = 0
        while stack:
            r, c = stack.pop()
            if (r, c) in visited:
                continue
            visited.add((r, c))
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nr, nc = r + dr, c + dc
                if 0 <= nr < self.size and 0 <= nc < self.size and self.board[nr, nc] == '.':
                    liberties += 1
        return liberties

    def evaluate(self):
        liberties = self.count_liberties(*self.atari_position)
        if liberties == 0:
            return -1  # White loses, no liberties
        elif liberties >= 4:
            return 1   # White wins, 4 or more liberties
        return liberties / 4 - 0.5  # Heuristic: normalize and shift for game state evaluation

    def is_game_over(self):
        liberties = self.count_liberties(*self.atari_position)
        if liberties == 0 or liberties >= 4:
            return True
        return False

    def reverse_colors(self):
        """Reverse the colors on the board."""
        for r in range(self.size):
            for c in range(self.size):
                if self.board[r, c] == 'B':
                    self.board[r, c] = 'W'
                elif self.board[r, c] == 'W':
                    self.board[r, c] = 'B'

    def to_sgf(self):
        to_play = self.board[self.atari_position[0], self.atari_position[1]]
        sgf_str = f"(;GM[1]FF[4]CA[UTF-8]SZ[19]AP[LadderPuzzle:1]PL[{to_play}]"

        black_positions = []
        white_positions = []
        for r in range(self.size):
            for c in range(self.size):
                pos = f"[{chr(c + 97)}{chr(r + 97)}]"
                if self.board[r, c] == 'B':
                    black_positions.append(pos)
                elif self.board[r, c] == 'W':
                    white_positions.append(pos)

        # Add all black and white stones at once
        if black_positions:
            sgf_str += f";AB{''.join(black_positions)}"
        if white_positions:
            sgf_str += f";AW{''.join(white_positions)}"

        # Mark the initial white stone with a square
        if self.atari_position:
            r, c = self.atari_position
            sgf_str += f";SQ[{chr(c + 97)}{chr(r + 97)}]"

        sgf_str += ")\n"
        return sgf_str


In [41]:
def check_valid_puzzle(board):
    # Check if the initial white stone's group is in atari and not captured
    if board.count_liberties(*board.atari_position) == 0:
        return False  # White stone is captured

    # Check entire board for any other stones in atari
    for row in range(board.size):
        for col in range(board.size):
            if (row, col) != board.atari_position and board.board[row, col] != '.':
                if board.count_liberties(row, col) == 1:
                    return False  # Another stone is in atari

    return True  # The puzzle is valid if no other groups are in atari and the initial white stone is not captured


In [42]:

def minimax(board, depth, maximizingPlayer):
    if depth == 0 or board.is_game_over():
        return board.evaluate(), depth

    if maximizingPlayer:
        maxEval = float('-inf')
        maxDepth = depth
        for row in range(board.size):
            for col in range(board.size):
                if board.place_stone(row, col, 'W'):
                    eval, current_depth = minimax(copy.deepcopy(board), depth - 1, False)
                    board.board[row, col] = '.'
                    if eval > maxEval:
                        maxEval = eval
                        maxDepth = current_depth
        return maxEval, maxDepth
    else:
        minEval = float('inf')
        minDepth = depth
        for row in range(board.size):
            for col in range(board.size):
                if board.place_stone(row, col, 'B'):
                    eval, current_depth = minimax(copy.deepcopy(board), depth - 1, True)
                    board.board[row, col] = '.'
                    if eval < minEval:
                        minEval = eval
                        minDepth = current_depth
        return minEval, minDepth



In [43]:
def generate_atari_puzzles(num_puzzles=10):
    for _ in range(num_puzzles):
        seed = np.random.randint(0, 999999)
        np.random.seed(seed)
        valid_setup = False
        while not valid_setup:
            board = GoBoard()
            board.place_stone_in_atari()
            board.random_fill(50)
            valid_setup = check_valid_puzzle(board)
#        _, complexity = minimax(board, 10, False)  # Assuming we start with Black to move
        if random.choice(['black', 'white']) == 'black':
            board.reverse_colors()
        sgf_output = board.to_sgf()
        filename = os.path.join(base_path,f"atari_puzzle_seed_{seed}.sgf")
        with open(filename, "w") as file:
            file.write(sgf_output)
        print(f"SGF for seed {seed} saved.")

generate_atari_puzzles()


SGF for seed 865392 saved.
SGF for seed 619037 saved.
SGF for seed 830070 saved.
SGF for seed 401844 saved.
SGF for seed 959681 saved.
SGF for seed 743221 saved.
SGF for seed 106123 saved.
SGF for seed 174005 saved.
SGF for seed 212427 saved.
SGF for seed 40750 saved.


In [20]:
!ls drive/MyDrive/auto_puzzle/sgf


atari_puzzle_seed_104701.sgf  atari_puzzle_seed_331233.sgf  atari_puzzle_seed_708028.sgf
atari_puzzle_seed_137792.sgf  atari_puzzle_seed_370182.sgf  atari_puzzle_seed_72118.sgf
atari_puzzle_seed_157935.sgf  atari_puzzle_seed_441490.sgf  atari_puzzle_seed_724771.sgf
atari_puzzle_seed_161351.sgf  atari_puzzle_seed_445981.sgf  atari_puzzle_seed_728172.sgf
atari_puzzle_seed_208206.sgf  atari_puzzle_seed_457368.sgf  atari_puzzle_seed_745781.sgf
atari_puzzle_seed_208648.sgf  atari_puzzle_seed_495146.sgf  atari_puzzle_seed_76365.sgf
atari_puzzle_seed_218201.sgf  atari_puzzle_seed_502230.sgf  atari_puzzle_seed_765766.sgf
atari_puzzle_seed_218228.sgf  atari_puzzle_seed_531285.sgf  atari_puzzle_seed_793134.sgf
atari_puzzle_seed_244330.sgf  atari_puzzle_seed_558036.sgf  atari_puzzle_seed_806329.sgf
atari_puzzle_seed_250533.sgf  atari_puzzle_seed_559376.sgf  atari_puzzle_seed_824112.sgf
atari_puzzle_seed_254454.sgf  atari_puzzle_seed_56946.sgf   atari_puzzle_seed_824681.sgf
atari_puzzle_seed_26554